# BATCH 188
##EMP ID :-4876
##Name :- T.manohar
##MLSPARK ASSIGNMENT

#Importing necessary libraries, mount to drive and importing File

In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("taitanic_model").getOrCreate()

In [ ]:
from pyspark.sql.types import *

# df_Schema describes the structure of the data in the train.csv file

df_Schema = StructType([
  StructField("PassengerId", StringType(), False),
  StructField("Survived", StringType(), False),
  StructField("Pclass", StringType(), False),
  StructField("Name", StringType(), False),
  StructField("Sex", StringType(), False),
  StructField("Age", IntegerType(), False),
  StructField("SibSp", StringType(), False),
  StructField("Parch", StringType(), False),
  StructField("Ticket", StringType(), False),
  StructField("Fare", IntegerType(), False),
  StructField("Cabin", IntegerType(), False),
  StructField("Embarked", StringType(), False)
])
#

In [ ]:
pwd

'/content'

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/bigdata2/assignment/train.csv",schema = df_Schema,header = True )

In [ ]:
df = df.repartition(32)
df.rdd.getNumPartitions()

32

In [ ]:
df.dtypes

[('PassengerId', 'string'),
 ('Survived', 'string'),
 ('Pclass', 'string'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'int'),
 ('SibSp', 'string'),
 ('Parch', 'string'),
 ('Ticket', 'string'),
 ('Fare', 'int'),
 ('Cabin', 'int'),
 ('Embarked', 'string')]

In [ ]:
df.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: integer (nullable = true)
 |-- Cabin: integer (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+------------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|            Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------------------+----+-----+--------+
|        834|       0|     3|Augustsson, Mr. A...|  male|  23|    0|    0|            347468|null| null|       S|
|        785|       0|     3|    Ali, Mr. William|  male|  25|    0|    0|SOTON/O.Q. 3101312|null| null|       S|
|        155|       0|     3|Olsen, Mr. Ole Ma...|  male|null|    0|    0|         Fa 265302|null| null|       S|
|        250|       0|     2|Carter, Rev. Erne...|  male|  54|    1|    0|            244252|  26| null|       S|
|        655|       0|     3|"Hegarty, Miss. H...|female|  18|    0|    0|            365226|null| null|       Q|
|        233|       0|     2|Sjostedt, Mr. Ern...|  male|  59|    0|    0|            23

#1.In the dataframe im checking that how many persons are survived and not survived

In [ ]:
num_of_passengers_survived = df[df.Survived==1]

In [ ]:
num_of_passengers_survived.count()

342

In [ ]:
num_of_not_passengers_survived = df.count()-num_of_passengers_survived.count()

In [ ]:
num_of_not_passengers_survived

549

#2.the count of male and female passengers surviving

In [ ]:
df_male = df[df.Sex=="male"]
df_male.count()

577

In [ ]:
df_male_surviver = df_male[df_male.Survived==1]
df_male_surviver.count()


109

The male surviver are 109 out of 577

In [ ]:
df_female = df[df.Sex=="female"]
df_female.count()

314

In [ ]:
df_female_surviver = df_female[df_female.Survived==1]
df_female_surviver.count()

233

In [ ]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+------------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|            Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------------------+----+-----+--------+
|        834|       0|     3|Augustsson, Mr. A...|  male|  23|    0|    0|            347468|null| null|       S|
|        785|       0|     3|    Ali, Mr. William|  male|  25|    0|    0|SOTON/O.Q. 3101312|null| null|       S|
|        155|       0|     3|Olsen, Mr. Ole Ma...|  male|null|    0|    0|         Fa 265302|null| null|       S|
|        250|       0|     2|Carter, Rev. Erne...|  male|  54|    1|    0|            244252|  26| null|       S|
|        655|       0|     3|"Hegarty, Miss. H...|female|  18|    0|    0|            365226|null| null|       Q|
|        233|       0|     2|Sjostedt, Mr. Ern...|  male|  59|    0|    0|            23

The total num females survived are 223 from 314 females passengers




In [ ]:
groupBy_output=df.groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       0|  549|
|       1|  342|
+--------+-----+



In [ ]:
df.groupBy("Sex","Survived").count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|female|       0|   81|
|female|       1|  233|
|  male|       1|  109|
|  male|       0|  468|
+------+--------+-----+



#3.columns are irrelevant and can be dropped

From DataFrame we can drop Name,Ticket,cabin. Because name and id we can define by Id and cabin has lot of null values more than 50%

In [ ]:
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|202|    0|    0|     0| 730|  891|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [ ]:
df_1 = df.drop("Name","Ticket","cabin")

In [ ]:
df_1.show()

+-----------+--------+------+------+----+-----+-----+----+--------+
|PassengerId|Survived|Pclass|   Sex| Age|SibSp|Parch|Fare|Embarked|
+-----------+--------+------+------+----+-----+-----+----+--------+
|        669|       0|     3|  male|  43|    0|    0|null|       S|
|        408|       1|     2|  male|   3|    1|    1|null|       S|
|        807|       0|     1|  male|  39|    0|    0|   0|       S|
|        136|       0|     2|  male|  23|    0|    0|null|       C|
|        518|       0|     3|  male|null|    0|    0|null|       Q|
|        181|       0|     3|female|null|    8|    2|null|       S|
|        483|       0|     3|  male|  50|    0|    0|null|       S|
|         98|       1|     1|  male|  23|    0|    1|null|       C|
|        470|       1|     3|female|null|    2|    1|null|       C|
|        353|       0|     3|  male|  15|    1|    1|null|       C|
|        535|       0|     3|female|  30|    0|    0|null|       S|
|        344|       0|     2|  male|  25|    0| 

In [ ]:
df_1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_1.columns]
   ).show()

+-----------+--------+------+---+---+-----+-----+----+--------+
|PassengerId|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|Embarked|
+-----------+--------+------+---+---+-----+-----+----+--------+
|          0|       0|     0|  0|202|    0|    0| 730|       2|
+-----------+--------+------+---+---+-----+-----+----+--------+



Im droping the Fare column because it has 50% above null values   

In [ ]:
df_1=df_1.drop("Fare")

In [ ]:
df_1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_1.columns]
   ).show()

+-----------+--------+------+---+---+-----+-----+--------+
|PassengerId|Survived|Pclass|Sex|Age|SibSp|Parch|Embarked|
+-----------+--------+------+---+---+-----+-----+--------+
|          0|       0|     0|  0|202|    0|    0|       2|
+-----------+--------+------+---+---+-----+-----+--------+



Im kepping id in variable for safty and im droping it

In [ ]:
df_ID = df_1.select("PassengerId")


In [ ]:
df_ID.show()

+-----------+
|PassengerId|
+-----------+
|        605|
|        356|
|        192|
|         46|
|         78|
|        446|
|        831|
|        161|
|        794|
|        306|
|        771|
|        287|
|        604|
|         80|
|        460|
|        479|
|        607|
|        859|
|        476|
|         65|
+-----------+
only showing top 20 rows



In [ ]:
df_1 = df_1.drop("PassengerId")

In [ ]:
df_1.show()

+--------+------+------+----+-----+-----+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|Embarked|
+--------+------+------+----+-----+-----+--------+
|       0|     3|  male|  30|    0|    0|       C|
|       1|     1|female|  39|    1|    1|       C|
|       0|     3|  male|null|    1|    0|       Q|
|       0|     3|  male|null|    0|    0|       C|
|       1|     3|female|null|    0|    0|       Q|
|       1|     1|female|  60|    1|    0|       C|
|       0|     3|  male|  16|    1|    1|       S|
|       0|     3|female|  21|    0|    0|       Q|
|       0|     3|  male|null|    0|    0|       S|
|       0|     1|female|   2|    1|    2|       S|
|       0|     3|female|  17|    0|    0|       C|
|       0|     1|  male|  61|    0|    0|       S|
|       1|     3|  male|null|    0|    0|       S|
|       0|     3|  male|  32|    0|    0|       Q|
|       1|     1|female|  21|    0|    0|       S|
|       1|     2|  male|   1|    2|    1|       S|
|       0|     1|  male|  54|  

In [ ]:
#null values after droping columns
df_1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_1.columns]
   ).show()


+--------+------+---+---+-----+-----+--------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Embarked|
+--------+------+---+---+-----+-----+--------+
|       0|     0|  0|202|    0|    0|       2|
+--------+------+---+---+-----+-----+--------+



In [ ]:
df_1.groupBy("Embarked").count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+



In Embarked the "s" is the most repeted so im filling with that in null values

In [ ]:
df_1 = df_1.na.fill({"Embarked":"S"})

In [ ]:
df_1.describe().show()

+-------+-------------------+-----------------+------+------------------+------------------+-------------------+--------+
|summary|           Survived|           Pclass|   Sex|               Age|             SibSp|              Parch|Embarked|
+-------+-------------------+-----------------+------+------------------+------------------+-------------------+--------+
|  count|                891|              891|   891|               689|               891|                891|     891|
|   mean| 0.3838383838383838|2.308641975308642|  null|29.847605224963715|0.5230078563411896|0.38159371492704824|    null|
| stddev|0.48659245426485753|0.836071240977049|  null|14.317668714749667|1.1027434322934317| 0.8060572211299483|    null|
|    min|                  0|                1|female|                 1|                 0|                  0|       C|
|    max|                  1|                3|  male|                80|                 8|                  6|       S|
+-------+---------------

Im filling the age null values with mean

In [ ]:
from pyspark.ml.feature import Imputer
imputer = Imputer(strategy='mean', inputCols=['Age'], outputCols=['AgeImputed'])
imputer_model = imputer.fit(df_1)
df_1 = imputer_model.transform(df_1)

In [ ]:
df_1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_1.columns]
   ).show()

+--------+------+---+---+-----+-----+--------+----------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Embarked|AgeImputed|
+--------+------+---+---+-----+-----+--------+----------+
|       0|     0|  0|202|    0|    0|       0|         0|
+--------+------+---+---+-----+-----+--------+----------+



In [ ]:
#im droping age beacuse ageImputed add as exctra column ill covert ageImputed to age

In [ ]:
df_1 = df_1.drop("Age")
df_1.show()

+--------+------+------+-----+-----+--------+----------+
|Survived|Pclass|   Sex|SibSp|Parch|Embarked|AgeImputed|
+--------+------+------+-----+-----+--------+----------+
|       0|     3|  male|    0|    0|       C|        30|
|       1|     1|female|    1|    1|       C|        39|
|       0|     3|  male|    1|    0|       Q|        29|
|       0|     3|  male|    0|    0|       C|        29|
|       1|     3|female|    0|    0|       Q|        29|
|       1|     1|female|    1|    0|       C|        60|
|       0|     3|  male|    1|    1|       S|        16|
|       0|     3|female|    0|    0|       Q|        21|
|       0|     3|  male|    0|    0|       S|        29|
|       0|     1|female|    1|    2|       S|         2|
|       0|     3|female|    0|    0|       C|        17|
|       0|     1|  male|    0|    0|       S|        61|
|       1|     3|  male|    0|    0|       S|        29|
|       0|     3|  male|    0|    0|       Q|        32|
|       1|     1|female|    0| 

In [ ]:
df_1 = df_1.withColumnRenamed("AgeImputed","Age")

In [ ]:
df_1.show()

+--------+------+------+-----+-----+--------+---+
|Survived|Pclass|   Sex|SibSp|Parch|Embarked|Age|
+--------+------+------+-----+-----+--------+---+
|       0|     3|  male|    0|    0|       C| 30|
|       1|     1|female|    1|    1|       C| 39|
|       0|     3|  male|    1|    0|       Q| 29|
|       0|     3|  male|    0|    0|       C| 29|
|       1|     3|female|    0|    0|       Q| 29|
|       1|     1|female|    1|    0|       C| 60|
|       0|     3|  male|    1|    1|       S| 16|
|       0|     3|female|    0|    0|       Q| 21|
|       0|     3|  male|    0|    0|       S| 29|
|       0|     1|female|    1|    2|       S|  2|
|       0|     3|female|    0|    0|       C| 17|
|       0|     1|  male|    0|    0|       S| 61|
|       1|     3|  male|    0|    0|       S| 29|
|       0|     3|  male|    0|    0|       Q| 32|
|       1|     1|female|    0|    0|       S| 21|
|       1|     2|  male|    2|    1|       S|  1|
|       0|     1|  male|    0|    1|       S| 54|


In [ ]:
df_1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_1.columns]
   ).show()

+--------+------+---+-----+-----+--------+---+
|Survived|Pclass|Sex|SibSp|Parch|Embarked|Age|
+--------+------+---+-----+-----+--------+---+
|       0|     0|  0|    0|    0|       0|  0|
+--------+------+---+-----+-----+--------+---+



Lets convert  columns from string to number using StringIndexer

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

In [ ]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df_1) for column in ["Sex","Embarked"]]
pipeline = Pipeline(stages=indexers)
df_train = pipeline.fit(df_1).transform(df_1)

In [ ]:
df_train.show()

+--------+------+------+-----+-----+--------+---+---------+--------------+
|Survived|Pclass|   Sex|SibSp|Parch|Embarked|Age|Sex_index|Embarked_index|
+--------+------+------+-----+-----+--------+---+---------+--------------+
|       0|     3|  male|    0|    0|       C| 30|      0.0|           1.0|
|       1|     1|female|    1|    1|       C| 39|      1.0|           1.0|
|       0|     3|  male|    1|    0|       Q| 29|      0.0|           2.0|
|       0|     3|  male|    0|    0|       C| 29|      0.0|           1.0|
|       1|     3|female|    0|    0|       Q| 29|      1.0|           2.0|
|       1|     1|female|    1|    0|       C| 60|      1.0|           1.0|
|       0|     3|  male|    1|    1|       S| 16|      0.0|           0.0|
|       0|     3|female|    0|    0|       Q| 21|      1.0|           2.0|
|       0|     3|  male|    0|    0|       S| 29|      0.0|           0.0|
|       0|     1|female|    1|    2|       S|  2|      1.0|           0.0|
|       0|     3|female| 

Lets drop extra columns

In [ ]:
df_train = df_train.drop("Sex","Embarked")

#4.Print the schema after data preprocessing

In [ ]:
df_train.printSchema()

root
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Sex_index: double (nullable = false)
 |-- Embarked_index: double (nullable = false)



Now converted str columns to int

In [ ]:
from pyspark.sql.types import IntegerType
df_train = df_train.withColumn("Pclass", df_train["Pclass"].cast(IntegerType()))
df_train = df_train.withColumn("SibSp", df_train["SibSp"].cast(IntegerType()))
df_train = df_train.withColumn("Parch", df_train["Parch"].cast(IntegerType()))
df_train = df_train.withColumn("Survived", df_train["Survived"].cast(IntegerType()))

now we pull all dependent variables to features

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
feature = VectorAssembler(inputCols=df_train.columns[1:],outputCol="features")
feature_vector= feature.transform(df_train)

In [ ]:
feature_vector.show()

+--------+------+-----+-----+---+---------+--------------+--------------------+
|Survived|Pclass|SibSp|Parch|Age|Sex_index|Embarked_index|            features|
+--------+------+-----+-----+---+---------+--------------+--------------------+
|       0|     3|    0|    0| 30|      0.0|           1.0|[3.0,0.0,0.0,30.0...|
|       1|     1|    1|    1| 39|      1.0|           1.0|[1.0,1.0,1.0,39.0...|
|       0|     3|    1|    0| 29|      0.0|           2.0|[3.0,1.0,0.0,29.0...|
|       0|     3|    0|    0| 29|      0.0|           1.0|[3.0,0.0,0.0,29.0...|
|       1|     3|    0|    0| 29|      1.0|           2.0|[3.0,0.0,0.0,29.0...|
|       1|     1|    1|    0| 60|      1.0|           1.0|[1.0,1.0,0.0,60.0...|
|       0|     3|    1|    1| 16|      0.0|           0.0|[3.0,1.0,1.0,16.0...|
|       0|     3|    0|    0| 21|      1.0|           2.0|[3.0,0.0,0.0,21.0...|
|       0|     3|    0|    0| 29|      0.0|           0.0|(6,[0,3],[3.0,29.0])|
|       0|     1|    1|    2|  2|      1

Now that the data is all set, let's split it into training and test. I'll be using 80% of it

In [ ]:
(trainingData, testData) = feature_vector.randomSplit([0.8, 0.2],seed = 11)

In [ ]:
trainingData.count()

724

In [ ]:
testData.count()

167

#5.Build and evaluate these models to classify (survived vs not survived)-
    1.Logistics Regression model
    2.Decision Tree classifier
    3.Random forest classifier
    4.Naive Bayes classifier

##1.logistic regresstion

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="Survived", featuresCol="features")
#Training algo
lrModel = lr.fit(trainingData)
lr_prediction = lrModel.transform(testData)
lr_prediction.select("prediction", "Survived", "features").show()
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[2.0,1.0,2.0,25.0...|
|       1.0|       0|[3.0,0.0,0.0,17.0...|
|       0.0|       0|[3.0,0.0,0.0,29.0...|
|       0.0|       0|[3.0,0.0,0.0,30.0...|
|       1.0|       0|[3.0,0.0,0.0,30.0...|
|       0.0|       0|[3.0,1.0,0.0,29.0...|
|       0.0|       0|[3.0,1.0,0.0,29.0...|
|       1.0|       1|[1.0,0.0,1.0,22.0...|
|       1.0|       1|[3.0,0.0,2.0,24.0...|
|       0.0|       0|(6,[0,3],[2.0,26.0])|
|       0.0|       0|(6,[0,3],[3.0,26.0])|
|       0.0|       0|[3.0,4.0,1.0,7.0,...|
|       1.0|       1|[2.0,0.0,0.0,23.0...|
|       0.0|       1|[2.0,1.0,1.0,2.0,...|
|       1.0|       1|[3.0,0.0,0.0,29.0...|
|       0.0|       0|[1.0,1.0,0.0,50.0...|
|       0.0|       0|(6,[0,3],[2.0,39.0])|
|       0.0|       0|[3.0,0.0,0.0,29.0...|
|       0.0|       1|[2.0,0.0,2.0,1.0,...|
|       1.0|       1|[2.0,1.0,2.0,5.0,...|
+----------

In [ ]:
lr_accuracy = evaluator.evaluate(lr_prediction)
print("Accuracy of LogisticRegression is = %g"% (lr_accuracy))

Accuracy of LogisticRegression is = 0.790419


##2.DecisionTreeClassifier

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="Survived", featuresCol="features",maxDepth=5,
                           minInstancesPerNode = 20, impurity = 'gini')
dt_model = dt.fit(trainingData)
dt_prediction = dt_model.transform(testData)
dt_prediction.select("prediction", "Survived", "features").show()

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[2.0,1.0,2.0,25.0...|
|       1.0|       0|[3.0,0.0,0.0,17.0...|
|       0.0|       0|[3.0,0.0,0.0,29.0...|
|       0.0|       0|[3.0,0.0,0.0,30.0...|
|       0.0|       0|[3.0,0.0,0.0,30.0...|
|       0.0|       0|[3.0,1.0,0.0,29.0...|
|       0.0|       0|[3.0,1.0,0.0,29.0...|
|       1.0|       1|[1.0,0.0,1.0,22.0...|
|       0.0|       1|[3.0,0.0,2.0,24.0...|
|       0.0|       0|(6,[0,3],[2.0,26.0])|
|       0.0|       0|(6,[0,3],[3.0,26.0])|
|       0.0|       0|[3.0,4.0,1.0,7.0,...|
|       1.0|       1|[2.0,0.0,0.0,23.0...|
|       0.0|       1|[2.0,1.0,1.0,2.0,...|
|       1.0|       1|[3.0,0.0,0.0,29.0...|
|       0.0|       0|[1.0,1.0,0.0,50.0...|
|       0.0|       0|(6,[0,3],[2.0,39.0])|
|       0.0|       0|[3.0,0.0,0.0,29.0...|
|       0.0|       1|[2.0,0.0,2.0,1.0,...|
|       1.0|       1|[2.0,1.0,2.0,5.0,...|
+----------

In [ ]:
dt_accuracy = evaluator.evaluate(dt_prediction)
print("Accuracy of DecisionTreeClassifier is = %g"% (dt_accuracy))

Accuracy of DecisionTreeClassifier is = 0.772455


##3.RandomForestClassifier

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="Survived", featuresCol="features")
rf_model = rf.fit(trainingData)
rf_prediction = rf_model.transform(testData)
rf_prediction.select("prediction", "Survived", "features").show()

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[2.0,1.0,2.0,25.0...|
|       1.0|       0|[3.0,0.0,0.0,17.0...|
|       0.0|       0|[3.0,0.0,0.0,29.0...|
|       0.0|       0|[3.0,0.0,0.0,30.0...|
|       1.0|       0|[3.0,0.0,0.0,30.0...|
|       0.0|       0|[3.0,1.0,0.0,29.0...|
|       0.0|       0|[3.0,1.0,0.0,29.0...|
|       1.0|       1|[1.0,0.0,1.0,22.0...|
|       0.0|       1|[3.0,0.0,2.0,24.0...|
|       0.0|       0|(6,[0,3],[2.0,26.0])|
|       0.0|       0|(6,[0,3],[3.0,26.0])|
|       0.0|       0|[3.0,4.0,1.0,7.0,...|
|       1.0|       1|[2.0,0.0,0.0,23.0...|
|       1.0|       1|[2.0,1.0,1.0,2.0,...|
|       1.0|       1|[3.0,0.0,0.0,29.0...|
|       0.0|       0|[1.0,1.0,0.0,50.0...|
|       0.0|       0|(6,[0,3],[2.0,39.0])|
|       0.0|       0|[3.0,0.0,0.0,29.0...|
|       1.0|       1|[2.0,0.0,2.0,1.0,...|
|       1.0|       1|[2.0,1.0,2.0,5.0,...|
+----------

In [ ]:
rf_accuracy = evaluator.evaluate(rf_prediction)
print("Accuracy of RandomForestClassifier is = %g"% (rf_accuracy))

Accuracy of RandomForestClassifier is = 0.778443


##4.NaiveBayes

In [ ]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(labelCol="Survived", featuresCol="features")
nb_model = nb.fit(trainingData)
nb_prediction = nb_model.transform(testData)
nb_prediction.select("prediction", "Survived", "features").show()

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[2.0,1.0,2.0,25.0...|
|       1.0|       0|[3.0,0.0,0.0,17.0...|
|       0.0|       0|[3.0,0.0,0.0,29.0...|
|       0.0|       0|[3.0,0.0,0.0,30.0...|
|       1.0|       0|[3.0,0.0,0.0,30.0...|
|       0.0|       0|[3.0,1.0,0.0,29.0...|
|       0.0|       0|[3.0,1.0,0.0,29.0...|
|       1.0|       1|[1.0,0.0,1.0,22.0...|
|       1.0|       1|[3.0,0.0,2.0,24.0...|
|       0.0|       0|(6,[0,3],[2.0,26.0])|
|       0.0|       0|(6,[0,3],[3.0,26.0])|
|       0.0|       0|[3.0,4.0,1.0,7.0,...|
|       1.0|       1|[2.0,0.0,0.0,23.0...|
|       0.0|       1|[2.0,1.0,1.0,2.0,...|
|       1.0|       1|[3.0,0.0,0.0,29.0...|
|       0.0|       0|[1.0,1.0,0.0,50.0...|
|       0.0|       0|(6,[0,3],[2.0,39.0])|
|       0.0|       0|[3.0,0.0,0.0,29.0...|
|       0.0|       1|[2.0,0.0,2.0,1.0,...|
|       1.0|       1|[2.0,1.0,2.0,5.0,...|
+----------

In [ ]:
nb_accuracy = evaluator.evaluate(nb_prediction)
print("Accuracy of NaiveBayes is  = %g"% (nb_accuracy))

Accuracy of NaiveBayes is  = 0.778443


# Im predicting on test data

In [ ]:
df_test_Schema = StructType([
  StructField("PassengerId", StringType(), False),
  StructField("Pclass", StringType(), False),
  StructField("Name", StringType(), False),
  StructField("Sex", StringType(), False),
  StructField("Age", IntegerType(), False),
  StructField("SibSp", StringType(), False),
  StructField("Parch", StringType(), False),
  StructField("Ticket", StringType(), False),
  StructField("Fare", IntegerType(), False),
  StructField("Cabin", IntegerType(), False),
  StructField("Embarked", StringType(), False)
])

In [ ]:
df_test = spark.read.csv("/content/test.csv",schema = df_test_Schema,header = True )

In [ ]:
df_test = df_test.repartition(32)
df_test.rdd.getNumPartitions()

32

In [ ]:
df_test.dtypes

[('PassengerId', 'string'),
 ('Pclass', 'string'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'int'),
 ('SibSp', 'string'),
 ('Parch', 'string'),
 ('Ticket', 'string'),
 ('Fare', 'int'),
 ('Cabin', 'int'),
 ('Embarked', 'string')]

In [ ]:
df_test.limit(2).show()

+-----------+------+--------------------+------+---+-----+-----+----------+----+-----+--------+
|PassengerId|Pclass|                Name|   Sex|Age|SibSp|Parch|    Ticket|Fare|Cabin|Embarked|
+-----------+------+--------------------+------+---+-----+-----+----------+----+-----+--------+
|        922|     2|Louch, Mr. Charle...|  male| 50|    1|    0|SC/AH 3085|  26| null|       S|
|        955|     3|Bradley, Miss. Br...|female| 22|    0|    0|    334914|null| null|       Q|
+-----------+------+--------------------+------+---+-----+-----+----------+----+-----+--------+



In [ ]:
df_test.count()

418

In [ ]:
#Droping cloumns which are droped in train data
df_test = df_test.drop("Name","Ticket","cabin","Fare")

In [ ]:
from pyspark.sql.functions import col,isnan, when, count
df_test.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_test.columns]
   ).show()

+-----------+------+---+---+-----+-----+--------+
|PassengerId|Pclass|Sex|Age|SibSp|Parch|Embarked|
+-----------+------+---+---+-----+-----+--------+
|          0|     0|  0|106|    0|    0|       0|
+-----------+------+---+---+-----+-----+--------+



In [ ]:
# Taking passenger Id to know whom im predicting and droping passengerIDs
df_test_ID = df_test.select("PassengerId")
df_test_ID.limit(2).show()

+-----------+
|PassengerId|
+-----------+
|       1134|
|       1167|
+-----------+



In [ ]:
df_test = df_test.drop("PassengerId")

In [ ]:
df_test.describe().show()

+-------+------------------+------+------------------+------------------+------------------+--------+
|summary|            Pclass|   Sex|               Age|             SibSp|             Parch|Embarked|
+-------+------------------+------+------------------+------------------+------------------+--------+
|  count|               418|   418|               312|               418|               418|     418|
|   mean|2.2655502392344498|  null|30.810897435897434|0.4473684210526316|0.3923444976076555|    null|
| stddev|0.8418375519640503|  null|13.869609024680017|0.8967595611217135|0.9814288785371693|    null|
|    min|                 1|female|                 1|                 0|                 0|       C|
|    max|                 3|  male|                76|                 8|                 9|       S|
+-------+------------------+------+------------------+------------------+------------------+--------+



In [ ]:
from pyspark.ml.feature import Imputer
imputer = Imputer(strategy='mean', inputCols=['Age'], outputCols=['age'])
imputer_model = imputer.fit(df_test)
df_test = imputer_model.transform(df_test)


In [ ]:
df_test.limit(2).show()

+------+----+---+-----+-----+--------+
|Pclass| Sex|age|SibSp|Parch|Embarked|
+------+----+---+-----+-----+--------+
|     2|male| 32|    0|    0|       S|
|     1|male| 67|    1|    0|       S|
+------+----+---+-----+-----+--------+



In [ ]:
df_test.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_test.columns]
   ).show()

+------+---+---+-----+-----+--------+
|Pclass|Sex|age|SibSp|Parch|Embarked|
+------+---+---+-----+-----+--------+
|     0|  0|  0|    0|    0|       0|
+------+---+---+-----+-----+--------+



In [ ]:
df_test.printSchema()

root
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df_test) for column in ["Sex","Embarked"]]
pipeline = Pipeline(stages=indexers)
df_test = pipeline.fit(df_test).transform(df_test)

In [ ]:
df_test.limit(2).show()

+------+----+---+-----+-----+--------+---------+--------------+
|Pclass| Sex|age|SibSp|Parch|Embarked|Sex_index|Embarked_index|
+------+----+---+-----+-----+--------+---------+--------------+
|     2|male| 32|    0|    0|       S|      0.0|           0.0|
|     1|male| 67|    1|    0|       S|      0.0|           0.0|
+------+----+---+-----+-----+--------+---------+--------------+



In [ ]:
df_test = df_test.drop("Sex","Embarked")

In [ ]:
df_test.limit(2).show()

+------+---+-----+-----+---------+--------------+
|Pclass|age|SibSp|Parch|Sex_index|Embarked_index|
+------+---+-----+-----+---------+--------------+
|     2| 32|    0|    0|      0.0|           0.0|
|     1| 67|    1|    0|      0.0|           0.0|
+------+---+-----+-----+---------+--------------+



In [ ]:
df_test.printSchema()

root
 |-- Pclass: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Sex_index: double (nullable = false)
 |-- Embarked_index: double (nullable = false)



In [ ]:
# Converting Str to int
from pyspark.sql.types import IntegerType
df_test = df_test.withColumn("Pclass", df_test["Pclass"].cast(IntegerType()))
df_test = df_test.withColumn("SibSp", df_test["SibSp"].cast(IntegerType()))
df_test = df_test.withColumn("Parch", df_test["Parch"].cast(IntegerType()))

In [ ]:
df_test.printSchema()

root
 |-- Pclass: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Sex_index: double (nullable = false)
 |-- Embarked_index: double (nullable = false)



In [ ]:
df_test.columns[:-1]

['Pclass', 'age', 'SibSp', 'Parch', 'Sex_index']

In [ ]:
feature = VectorAssembler(inputCols=df_test.columns,outputCol="features")
df_test_feature_vector= feature.transform(df_test)

In [ ]:
df_test_feature_vector.show()

+------+---+-----+-----+---------+--------------+--------------------+
|Pclass|age|SibSp|Parch|Sex_index|Embarked_index|            features|
+------+---+-----+-----+---------+--------------+--------------------+
|     2| 32|    0|    0|      0.0|           0.0|(6,[0,1],[2.0,32.0])|
|     1| 67|    1|    0|      0.0|           0.0|[1.0,67.0,1.0,0.0...|
|     3| 30|    1|    2|      0.0|           0.0|[3.0,30.0,1.0,2.0...|
|     2| 24|    2|    0|      0.0|           0.0|[2.0,24.0,2.0,0.0...|
|     3| 50|    1|    0|      0.0|           0.0|[3.0,50.0,1.0,0.0...|
|     3| 21|    1|    0|      0.0|           0.0|[3.0,21.0,1.0,0.0...|
|     1| 39|    1|    0|      0.0|           1.0|[1.0,39.0,1.0,0.0...|
|     3| 30|    0|    0|      1.0|           2.0|[3.0,30.0,0.0,0.0...|
|     3| 23|    0|    0|      1.0|           0.0|[3.0,23.0,0.0,0.0...|
|     3| 32|    0|    0|      0.0|           0.0|(6,[0,1],[3.0,32.0])|
|     1| 21|    0|    1|      0.0|           1.0|[1.0,21.0,0.0,1.0...|
|     

Im using logistic because from above models it got highest accuracy from another models

In [ ]:

lr_test_prediction = lrModel.transform(df_test_feature_vector)
lr_test_prediction.select("prediction").show()


+----------+
|prediction|
+----------+
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
+----------+
only showing top 20 rows



In [ ]:
df_test.count()

418

In [ ]:
lr_test_prediction.count()

418

In [ ]:
pwd

'/content'

In [ ]:
final_prediction = lr_test_prediction.toPandas()

In [ ]:
final_prediction.head(2)

,Pclass,age,SibSp,Parch,Sex_index,Embarked_index,features,rawPrediction,probability,prediction
0,2,32,0,0,0.0,0.0,"(2.0, 32.0, 0.0, 0.0, 0.0, 0.0)","[8.456550090156533, -8.456550090156533]","[0.9997875412214067, 0.0002124587785933274]",0.0
1,1,67,1,0,0.0,0.0,"[1.0, 67.0, 1.0, 0.0, 0.0, 0.0]","[16.35546227081244, -16.35546227081244]","[0.9999999211297981, 7.88702019249854e-08]",0.0


In [ ]:
final_prediction.to_csv("ml_spark_final_predictions.csv")

